In [ ]:
!pip install tensorflow numpy pandas seaborn ucimlrepo

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow.keras as keras
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import TimeSeriesSplit
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, Conv1D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.activations import sigmoid
from ucimlrepo import fetch_ucirepo

---

# Dataset

In [ ]:
# fetch dataset
individual_household_electric_power_consumption = fetch_ucirepo(id=235)

# data (as pandas dataframes)
data = individual_household_electric_power_consumption.data.features

# metadata
print(individual_household_electric_power_consumption.metadata)

# variable information
print(individual_household_electric_power_consumption.variables)

In [ ]:
data.head(10)

In [ ]:
data['datetime'] = pd.to_datetime(data['Date'] + ' ' + data['Time'], format='%d/%m/%Y %H:%M:%S')

data = data.set_index('datetime').sort_index()

data.drop(['Date', 'Time'], axis=1, inplace=True)
data = data[data['Global_active_power'] != '?']

In [ ]:
datetimes = data.index.to_numpy()
gap = data['Global_active_power'].to_numpy().astype(float)

### Plotting data

In [ ]:
fig, ax = plt.subplots(figsize=(15, 5))
ax.plot(gap[:500])

---

# Neural Network

In [ ]:
early_stopping = EarlyStopping(
    monitor="val_loss",
    patience=3,
    restore_best_weights=True,
)

In [ ]:
def create_sliding_window_data(data, window_size):
    X = []
    y = []
    for i in range(len(data) - window_size):
        X.append(data[i:i + window_size])  # The window of input data
        y.append(data[i + window_size])    # The next value as the target

    return np.array(X), np.array(y)

n_splits = 5
window_size = 100

tscv = TimeSeriesSplit(n_splits=n_splits)
for i, (train_index, test_index) in enumerate(tscv.split(gap)):
    #To get the indices
    gap_train, gap_test = gap[train_index], gap[test_index]
    X_train, y_train = create_sliding_window_data(gap_train, window_size)
    X_test, y_test = create_sliding_window_data(gap_test, window_size)

    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=5, activation='relu', input_shape=(window_size, 1)))
    model.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(30, activation='relu'))
    model.add(Dense(1))  # Output layer for regression
    model.compile(optimizer='adam', loss='mse')

    history = model.fit(
        x=X_train,
        y=y_train,
        batch_size=50,
        epochs=50,
        shuffle=True,
        validation_data=(X_test, y_test),
        callbacks=[early_stopping],
    )

    results = model.predict(X_test).reshape(-1)

    fig, axs = plt.subplots(ncols=1, nrows=2, constrained_layout=True, figsize=(20,5))
    fig.suptitle(f'Fold {i+1}')
    axs[0].set_title('Train Test Split')
    axs[0].plot(datetimes[train_index], gap_train, c='blue', label='train')
    axs[0].plot(datetimes[test_index], gap_test, c='orange', label='test')
    axs[0].legend()
    axs[1].set_title('test result')
    axs[1].plot(datetimes[test_index][window_size:], y_test, c='blue', label='ground truth')
    axs[1].plot(datetimes[test_index][window_size:], results, c='orange', label='results')
    axs[1].legend()
    plt.show()

In [ ]:
history = model.fit(
    x=x_train,
    y=y_train_oh,
    batch_size=50,
    epochs=100,
    shuffle=True,
    validation_data=(x_test, y_test_oh),
    callbacks=[early_stopping],
)

In [ ]:
h_df = pd.DataFrame(history.history)

metrics = set([k.split('_')[-1] for k in history.history.keys()])

fig, axs = plt.subplots(ncols=len(metrics), nrows=1, constrained_layout=True, figsize=(20,5))

for plot_idx, metric in enumerate(metrics):
    best = h_df.max(0) if metric != 'loss' else h_df.min(0)
    axs[plot_idx].plot(
        history.history[metric],
        label=f'training (best={best[metric]:.3f})'
    )
    axs[plot_idx].plot(
        history.history['val_'+metric],
        label=f'validation (best={best["val_"+metric]:.3f})'
    )
    axs[plot_idx].grid(True)
    axs[plot_idx].set_title(metric)
    axs[plot_idx].legend()


In [ ]:
y_prediction = model.predict(x_test)
y_prediction = np.argmax(y_prediction, axis=1)

cf_matrix = confusion_matrix(y_test, y_prediction, normalize=None)

fig, ax = plt.subplots(figsize=(7,7))
sns.heatmap(cf_matrix, annot=True, fmt='', cmap='Blues', ax=ax)